In [1]:
## Essential Imports: 
import os
import numpy as np
from pathlib import Path
from pzflow.examples import get_galaxy_data
import tables_io

## RAIL-Specific Imports: 
import rail
from rail.creation.degradation import LSSTErrorModel, InvRedshiftIncompleteness
from rail.creation.engines.flowEngine import FlowModeler, FlowCreator, FlowPosterior
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

#from rail.estimation.algos.bpz_lite import Inform_BPZ_lite, BPZ_lite
#from rail.estimation.algos.knnpz import Inform_KNearNeighPDF, KNearNeighPDF
from rail.estimation.algos.flexzboost import Inform_FZBoost, FZBoost

#from rail.estimation.algos.naiveStack import NaiveStack
#from rail.estimation.algos.pointEstimateHist import PointEstimateHist

from rail.evaluation.evaluator import Evaluator

## Data Storage: 
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

## Data Formatting: (from different codes) 
from rail.core.utils import RAILDIR
flow_file = os.path.join(RAILDIR, 'examples/goldenspike/data/pretrained_flow.pkl')
bands = ['u','g','r','i','z','y']
band_dict = {band:f'mag_{band}_lsst' for band in bands}
rename_dict = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

# 100,000-Galaxy Catalog

In [2]:
#path to access the data 
DATA_DIR = Path().resolve() / "data"
DATA_DIR.mkdir(exist_ok=True)

catalog_file = DATA_DIR / "base_catalog.pq"

#array of 100,000 galaxies w/ 7 attributes for each: redshift & ugrizy
catalog = get_galaxy_data().rename(band_dict, axis=1) 
print(catalog)

#turns array into a table 
tables_io.write(catalog, str(catalog_file.with_suffix("")), catalog_file.suffix[1:])

catalog_file = str(catalog_file)
flow_file = str(DATA_DIR / "trained_flow.pkl")


       redshift  mag_u_lsst  mag_g_lsst  mag_r_lsst  mag_i_lsst  mag_z_lsst  \
0      0.287087   26.759261   25.901778   25.187710   24.932318   24.736903   
1      0.293313   27.428358   26.679299   25.977161   25.700094   25.522763   
2      1.497276   27.294001   26.068798   25.450055   24.460507   23.887221   
3      0.283310   28.154075   26.283166   24.599570   23.723491   23.214108   
4      1.545183   29.276065   27.878301   27.333528   26.543374   26.061941   
...         ...         ...         ...         ...         ...         ...   
99995  0.661883   25.665363   25.703183   25.362422   24.936967   24.771529   
99996  0.864290   28.342505   27.654294   26.638448   25.615190   25.108037   
99997  0.973176   27.781264   27.386962   26.760144   26.107564   25.584028   
99998  2.281685   29.324894   28.148004   27.312318   26.883849   26.416446   
99999  0.254427   22.077757   20.985672   20.143253   19.805022   19.594315   

       mag_y_lsst  
0       24.671623  
1       25.

# 10,000-Galaxy Catalog 


In [3]:
###

# Degrade 100,000-Galaxy Catalog with LSST Photometric Error Model 

In [4]:
inv_redshift = InvRedshiftIncompleteness.make_stage(
    name='inv_redshift',
    pivot_redshift=1.0,
)

# Degrade 10,000-Galaxy Catalog with Redshift Incompleteness 

In [5]:
lsst_error_model_train = LSSTErrorModel.make_stage(
    name='lsst_error_model_train',
    bandNames=band_dict, 
    seed=29,
)

# Train FlexzBoost on Degraded Data 

In [6]:
###

# Perform Estimations with FlexzBoost

In [7]:
###

# Evaluate Using Performance Metrics 

In [8]:
###